In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

from tqdm import tqdm
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline



#### Preprocessing
- `box_dimension.csv`: 運送箱規格大小
    - 算出體積
- `sku_dimension.csv`: 商品大小
- `order_sku.csv`: 訂單含哪些商品
    - 和 `sku_dimension.csv` 合併，算出訂單最小長寬高及所需體積


#### Strategy 
Due to time limitation (2hr), we choose to split the problem to 3 parts:
1. the simplest case: 1-sku orders
2. 2nd simplest case: multiple 1-unique-sku orders
3. the rest of the case


In [11]:
box_dim = pd.read_csv('datasets/box_dimension.csv')
box_dim['volume'] = box_dim['length']*box_dim['width']*box_dim['height']
sku_dim = pd.read_csv('datasets/sku_dimension.csv')
orders = pd.read_csv('datasets/order_sku.csv')

box_dim.shape
box_dim.head()

sku_dim.shape
sku_dim.head()

orders.shape
orders.head()


(22, 6)

,id,box_number,length,width,height,volume
0,1,SP-1001,15.0,15.0,20.0,4500.0
1,2,SP-1002,20.0,20.0,32.0,12800.0
2,3,SP-1003,32.5,32.5,50.0,52812.5
3,4,SP-101,11.0,12.0,24.0,3168.0
4,5,SP-102,20.0,20.0,25.0,10000.0


(19216, 4)

,sku_id,length,width,height
0,0fa25745b0e1d8a63caacb7ea6639f6c825d96ca157bb7...,50.0,29.0,44.0
1,f8b3960858784b45e85ff7fed33d8538d711efcb6672c5...,10.0,5.0,3.0
2,50b33759532df7664e435eac02c62197304c80c13af3dd...,4.0,17.0,8.0
3,b398bd222151d4223576eba6d0ee3c2125f27f351336f6...,16.0,10.0,3.0
4,62983ad925d16e06ad8b5df2d29bb31be7efc571f2ae7c...,16.0,11.0,17.0


(181027, 3)

,order_number,sku_id,quantity
0,be35924799d0a1b179795420378981820722fd6681eed4...,0fa25745b0e1d8a63caacb7ea6639f6c825d96ca157bb7...,1
1,fa129fe24ae2fd26ff4ddd132a4f3b2564831e906dd10d...,f8b3960858784b45e85ff7fed33d8538d711efcb6672c5...,2
2,05c9ae40ad96a5ab0a527129afbf4a5b28d7bfc8200796...,50b33759532df7664e435eac02c62197304c80c13af3dd...,1
3,643b1bc23521958d9bdd730c85bef211ebc3d11bd93682...,b398bd222151d4223576eba6d0ee3c2125f27f351336f6...,1
4,ea816de880f43e51d95f542d3996a80636aa2e921d4acf...,62983ad925d16e06ad8b5df2d29bb31be7efc571f2ae7c...,1


#### 1. the simplest case: 1-sku orders

In [12]:
order_skus = orders.groupby(['order_number']).size().reset_index().rename(columns={0: 'sku_num'})
orderid_1sku = order_skus[order_skus['sku_num']==1].order_number.unique()
df_order_1sku = orders[orders['order_number'].isin(orderid_1sku)]
df_order_1sku = pd.merge(df_order_1sku, sku_dim, on='sku_id', how='left')
df_order_1sku_1quantity = df_order_1sku[df_order_1sku['quantity']==1]
df_order_1sku_1quantity['volume'] = df_order_1sku_1quantity['length']*df_order_1sku_1quantity['width']*df_order_1sku_1quantity['height']
df_order_1sku_1quantity

id_df_order_1sku_1quantity = df_order_1sku_1quantity.order_number.values


C:\Users\Administrator\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,order_number,sku_id,quantity,length,width,height,volume
0,be35924799d0a1b179795420378981820722fd6681eed4...,0fa25745b0e1d8a63caacb7ea6639f6c825d96ca157bb7...,1,50.0,29.0,44.0,63800.0
2,05c9ae40ad96a5ab0a527129afbf4a5b28d7bfc8200796...,50b33759532df7664e435eac02c62197304c80c13af3dd...,1,4.0,17.0,8.0,544.0
3,643b1bc23521958d9bdd730c85bef211ebc3d11bd93682...,b398bd222151d4223576eba6d0ee3c2125f27f351336f6...,1,16.0,10.0,3.0,480.0
4,3c802d480297fa530dc5b5fec5f433f889292dbf1b06cd...,aba4433c3997d1752172722dfd24d41f6defea8bfd6279...,1,10.0,10.0,3.3,330.0
5,6d247174bdc59099f90ff22883d0649aa3abadbbf7abbe...,0e05125eeef525f358130891d9703b7ee48d5fcc202800...,1,12.0,11.0,19.0,2508.0
...,...,...,...,...,...,...,...
66385,2bd4df2a9b53f37f50c44b761ca405ee51805824735406...,f76b3c8fd02f64f565bdedd067b59dde930f97cfbe8f90...,1,7.0,27.0,27.0,5103.0
66386,6d6761784c7a81c9ff3c74abadcb78ea700a7d784bead3...,6d0375ec573420c731aaed978c8d96d31acdc366d758c6...,1,12.0,5.0,1.0,60.0
66387,0e1c0ff1689f552bb713a071e98162c30e95ec6176b5d3...,9250c3c8d956feedded80c470bb31b98c036de5c99de96...,1,39.0,39.0,39.0,59319.0
66390,2d05395f7ed34f0379b442ceafc27295aeec662134e025...,f8b8c4e6f2868ebdeac01941a4609ec54f5ebe754f8a96...,1,25.7,29.0,42.0,31302.6


In [13]:
fitted_box_list = list()
for i in tqdm(range(df_order_1sku_1quantity.shape[0])):
    all_sku_dim = np.sort(df_order_1sku_1quantity.iloc[i,3:6].values) # 三邊長統一由小到大排序
    all_sku_vol = df_order_1sku_1quantity.iloc[i,6]
    box_vol_fitted = box_dim[box_dim['volume']>=all_sku_vol].sort_values(['volume','id'], ascending=[True, True])  # 符合大小的箱子由小到大排序
    
    each_order_qualify = list()
    for j in range(box_vol_fitted.shape[0]):
        
        pre_fit_box = np.sort(box_vol_fitted.iloc[j, 2:5].values) # 體積符合標準的箱子，三邊長統一由小到大排序後再次確認可否裝下
        
        if all(pre_fit_box > all_sku_dim) == True:
            fitted_box_id = box_vol_fitted.iloc[j, 1]
            each_order_qualify.append(fitted_box_id)
    
    if len(each_order_qualify) > 0:
        fitted_box_list.append(each_order_qualify[0])
    else:
        fitted_box_list.append('UNFITTED')

len(fitted_box_list)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 54892/54892 [04:23<00:00, 208.61it/s]


54892

In [18]:
df_order_1sku_1quantity['smallest_box_number_fitted'] = fitted_box_list
df_order_1sku_1quantity['id'] = fitted_box_list
df_order_1sku_1quantity = pd.merge(df_order_1sku_1quantity, box_dim[['id','box_number']], on='id', how='left')
df_order_1sku_1quantity


,order_number,sku_id,quantity,length,width,height,volume,smallest_box_number_fitted,id,box_number
0,be35924799d0a1b179795420378981820722fd6681eed4...,0fa25745b0e1d8a63caacb7ea6639f6c825d96ca157bb7...,1,50.0,29.0,44.0,63800.0,9,9,SP-106
1,05c9ae40ad96a5ab0a527129afbf4a5b28d7bfc8200796...,50b33759532df7664e435eac02c62197304c80c13af3dd...,1,4.0,17.0,8.0,544.0,11,11,SP-108
2,643b1bc23521958d9bdd730c85bef211ebc3d11bd93682...,b398bd222151d4223576eba6d0ee3c2125f27f351336f6...,1,16.0,10.0,3.0,480.0,11,11,SP-108
3,3c802d480297fa530dc5b5fec5f433f889292dbf1b06cd...,aba4433c3997d1752172722dfd24d41f6defea8bfd6279...,1,10.0,10.0,3.3,330.0,11,11,SP-108
4,6d247174bdc59099f90ff22883d0649aa3abadbbf7abbe...,0e05125eeef525f358130891d9703b7ee48d5fcc202800...,1,12.0,11.0,19.0,2508.0,1,1,SP-1001
...,...,...,...,...,...,...,...,...,...,...
54887,2bd4df2a9b53f37f50c44b761ca405ee51805824735406...,f76b3c8fd02f64f565bdedd067b59dde930f97cfbe8f90...,1,7.0,27.0,27.0,5103.0,18,18,SP-115
54888,6d6761784c7a81c9ff3c74abadcb78ea700a7d784bead3...,6d0375ec573420c731aaed978c8d96d31acdc366d758c6...,1,12.0,5.0,1.0,60.0,11,11,SP-108
54889,0e1c0ff1689f552bb713a071e98162c30e95ec6176b5d3...,9250c3c8d956feedded80c470bb31b98c036de5c99de96...,1,39.0,39.0,39.0,59319.0,9,9,SP-106
54890,2d05395f7ed34f0379b442ceafc27295aeec662134e025...,f8b8c4e6f2868ebdeac01941a4609ec54f5ebe754f8a96...,1,25.7,29.0,42.0,31302.6,7,7,SP-104


In [24]:
output_case1 = pd.DataFrame({
    'order_number': orders.order_number.unique()
})
output_case1 = pd.merge(output_case1, df_order_1sku_1quantity[['order_number', 'box_number']], on='order_number', how='left')
output_case1['box_number'] = output_case1['box_number'].fillna('UNFITTED')
output_case1.head()
# output_case1.to_csv('submission_1.csv', index=False)


,order_number,box_number
0,be35924799d0a1b179795420378981820722fd6681eed4...,SP-106
1,fa129fe24ae2fd26ff4ddd132a4f3b2564831e906dd10d...,UNFITTED
2,05c9ae40ad96a5ab0a527129afbf4a5b28d7bfc8200796...,SP-108
3,643b1bc23521958d9bdd730c85bef211ebc3d11bd93682...,SP-108
4,ea816de880f43e51d95f542d3996a80636aa2e921d4acf...,UNFITTED


#### 2. 2nd simplest case: multiple 1-unique-sku orders

In [22]:
df_order_1sku_several = df_order_1sku[df_order_1sku['quantity']>1]
# 用最小邊堆疊來節省裝箱空間
df_order_1sku_several['minimum_dim'] = df_order_1sku_several.iloc[:, 3:6].min(axis=1)
df_order_1sku_several['total_height'] = df_order_1sku_several['minimum_dim'] * df_order_1sku_several['quantity']
df_order_1sku_several.shape
df_order_1sku_several.head()


C:\Users\Administrator\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Administrator\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


(11500, 8)

,order_number,sku_id,quantity,length,width,height,minimum_dim,total_height
1,fa129fe24ae2fd26ff4ddd132a4f3b2564831e906dd10d...,f8b3960858784b45e85ff7fed33d8538d711efcb6672c5...,2,10.0,5.0,3.0,3.0,6.0
9,2a98e64d61d63381e0791501ba688e2fd64b67676e4b67...,79dc4543aa87f75717bd5026be2833ed5f8e4fbe28f317...,2,13.0,8.0,1.0,1.0,2.0
21,867943a7776e3a1ec682484761a171e5af6a7d49f30bb5...,dfe33bb2990ac9486b51f89a3d417545be614c060d8a44...,2,15.0,2.0,2.0,2.0,4.0
37,1d38fead305509f0fce933cafaf2b98b823a314cbfc45c...,38ce33dd1148cc77faf30154f81eaa738ffbdbcc09f0a3...,2,10.0,23.0,69.0,10.0,20.0
42,de88dcf2fe69d930f9874e3184b6df5e76119602037075...,7b9bbc25e346228c360c2dda71936557e94885ea516072...,2,15.0,9.0,1.0,1.0,2.0


In [23]:
fitted_box_list = list()
for i in tqdm(range(df_order_1sku_several.shape[0])):
    dims = df_order_1sku_several.iloc[i, 3:6].values.tolist()
    dims[dims.index(min(dims))] = min(dims)*df_order_1sku_several.iloc[i, 2] # 將最小邊乘以商品數
    dims = sorted(dims)
    volume = dims[0]*dims[1]*dims[2]
    box_vol_fitted = box_dim[box_dim['volume']>=volume].sort_values(['volume','id'], ascending=[True, True])

    each_order_qualify = list()
    for j in range(box_vol_fitted.shape[0]):

        pre_fit_box = np.sort(box_vol_fitted.iloc[j, 2:5].values)

        if all(pre_fit_box > dims) == True:
            fitted_box_num = box_vol_fitted.iloc[j, 1]
            each_order_qualify.append(fitted_box_num)

    if len(each_order_qualify)>0:
        fitted_box_list.append(each_order_qualify[0])
    else:
        fitted_box_list.append('UNFITTED')

len(fitted_box_list)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11500/11500 [01:01<00:00, 187.97it/s]


11500

In [35]:
df_order_1sku_several['smallest_box_number_fitted'] = fitted_box_list
output_case2 = pd.merge(output_case1, df_order_1sku_several[['order_number', 'smallest_box_number_fitted']], on='order_number', how='left')
output_case2['box_number'] = np.where(output_case2['box_number']=='UNFITTED', 
                                      output_case2['smallest_box_number_fitted'], 
                                      output_case2['box_number'])
output_case2 = output_case2[['order_number','box_number']]
output_case2['box_number'] = output_case2['box_number'].fillna('UNFITTED')
output_case2.head()
# output_case2.to_csv('submission_2.csv', index=False)


,order_number,box_number
0,be35924799d0a1b179795420378981820722fd6681eed4...,SP-106
1,fa129fe24ae2fd26ff4ddd132a4f3b2564831e906dd10d...,SP-108
2,05c9ae40ad96a5ab0a527129afbf4a5b28d7bfc8200796...,SP-108
3,643b1bc23521958d9bdd730c85bef211ebc3d11bd93682...,SP-108
4,ea816de880f43e51d95f542d3996a80636aa2e921d4acf...,UNFITTED


#### 3. the rest of cases
- 從最大的開始堆疊，每次堆完就算一次三邊長，再用它和第二大的最短邊堆疊，重複下去


In [53]:
orderid_multisku = order_skus[order_skus['sku_num']>1].order_number.unique()
df_order_multisku = orders[orders['order_number'].isin(orderid_multisku)]
df_order_multisku = pd.merge(df_order_multisku, sku_dim, on='sku_id', how='left')
df_order_multisku.head()


,order_number,sku_id,quantity,length,width,height
0,ea816de880f43e51d95f542d3996a80636aa2e921d4acf...,62983ad925d16e06ad8b5df2d29bb31be7efc571f2ae7c...,1,16.0,11.0,17.0
1,ea816de880f43e51d95f542d3996a80636aa2e921d4acf...,889d0409821df0281a127809f8a5c5b923bc67db29538d...,1,10.0,10.5,22.5
2,c5fbf7887ba485ce64383d86346a1777357a48a1137aa3...,78e3413f3adbd925cb5e16f4f9ec06e092118c83d1abfc...,1,10.0,4.0,15.0
3,c5fbf7887ba485ce64383d86346a1777357a48a1137aa3...,a43413e02dba3f91e702b58e1ec264df0b23993d52db40...,1,4.0,2.0,8.5
4,c5fbf7887ba485ce64383d86346a1777357a48a1137aa3...,e18b6d606f95aceba4ef7e58d24e47b12df8eb4c08d5c9...,1,5.0,5.0,3.0


In [54]:
# 先把重複商品堆疊後的最佳三邊更新

stacked_sidelengths = list()
for skuid in tqdm(df_order_multisku.sku_id.unique()):
    tmp = df_order_multisku[df_order_multisku['sku_id']==skuid]
    sidelength = tmp.iloc[0, 3:6].tolist()
    quan = tmp.iloc[0, 2]
    sidelength[sidelength.index(min(sidelength))] = quan*min(sidelength)
    stacked_sidelengths.append([skuid, sidelength[0], sidelength[1], sidelength[2]])

df_order_multisku = pd.merge(df_order_multisku, 
                             pd.DataFrame(stacked_sidelengths, columns=['sku_id','new_length','new_width','new_height']), 
                             on='sku_id', how='left')
df_order_multisku = df_order_multisku.drop(['length','width','height'], axis=1)
df_order_multisku = df_order_multisku.rename(columns={'new_length': 'length', 
                                                      'new_width': 'width', 
                                                      'new_height': 'height'})
df_order_multisku.head()


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15433/15433 [01:50<00:00, 139.66it/s]


,order_number,sku_id,quantity,length,width,height
0,ea816de880f43e51d95f542d3996a80636aa2e921d4acf...,62983ad925d16e06ad8b5df2d29bb31be7efc571f2ae7c...,1,16.0,11.0,17.0
1,ea816de880f43e51d95f542d3996a80636aa2e921d4acf...,889d0409821df0281a127809f8a5c5b923bc67db29538d...,1,10.0,10.5,22.5
2,c5fbf7887ba485ce64383d86346a1777357a48a1137aa3...,78e3413f3adbd925cb5e16f4f9ec06e092118c83d1abfc...,1,10.0,4.0,15.0
3,c5fbf7887ba485ce64383d86346a1777357a48a1137aa3...,a43413e02dba3f91e702b58e1ec264df0b23993d52db40...,1,4.0,2.0,8.5
4,c5fbf7887ba485ce64383d86346a1777357a48a1137aa3...,e18b6d606f95aceba4ef7e58d24e47b12df8eb4c08d5c9...,1,5.0,5.0,3.0


In [62]:
oid = 'ea816de880f43e51d95f542d3996a80636aa2e921d4acfa4388bdb95bf588da5'
packet_skus = df_order_multisku[df_order_multisku['order_number']==oid]
stack_order = packet_skus.iloc[:, 3:6].values.max(axis=1).tolist()
stack_order = pd.DataFrame({
    'index': list(range(len(stack_order))),
    'value': stack_order
}).sort_values('value').index.values.tolist()

last_sidelengths = list()
for i, row in enumerate(stack_order):
    if i == 0:
        last_sidelengths = packet_skus.iloc[row, 3:6]
        last_sidelengths = np.sort(last_sidelengths)  # 方便比對，三邊都從小排到大 
    else:
        last_sidelengths += np.array([np.sort(packet_skus.iloc[row, 3:6])[0], 0, 0])  # 第二個開始，只往最短邊堆疊
        last_sidelengths = np.sort(last_sidelengths)  # 每次堆完，三邊再次從小排到大(不會永遠只堆同一邊)



C:\Users\Administrator\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,order_number,sku_id,quantity,length,width,height,max_length
0,ea816de880f43e51d95f542d3996a80636aa2e921d4acf...,62983ad925d16e06ad8b5df2d29bb31be7efc571f2ae7c...,1,16.0,11.0,17.0,16.0
1,ea816de880f43e51d95f542d3996a80636aa2e921d4acf...,889d0409821df0281a127809f8a5c5b923bc67db29538d...,1,10.0,10.5,22.5,16.0


In [95]:
fitted_box_list = list()
for oid in tqdm(df_order_multisku.order_number.unique()):
    packet_skus = df_order_multisku[df_order_multisku['order_number']==oid]
    stack_order = packet_skus.iloc[:, 3:6].values.max(axis=1).tolist()
    
    # 根據生活經驗，從有最大邊長的商品開始堆疊
    stack_order = pd.DataFrame({
        'index': list(range(len(stack_order))),
        'value': stack_order
    }).sort_values('value').index.values.tolist()
    
    # 計算堆完的最佳三邊長
    last_sidelengths = list()
    for i, row in enumerate(stack_order):
        if i == 0:
            last_sidelengths = packet_skus.iloc[row, 3:6]
            last_sidelengths = np.sort(last_sidelengths)  # 方便比對，三邊都從小排到大 
        else:
            last_sidelengths += np.array([np.sort(packet_skus.iloc[row, 3:6])[0], 0, 0])  # 第二個開始，只往最短邊堆疊
            last_sidelengths = np.sort(last_sidelengths)  # 每次堆完，三邊再次從小排到大(不會永遠只堆同一邊)

    dims = last_sidelengths
    volume = dims[0]*dims[1]*dims[2]
    box_vol_fitted = box_dim[box_dim['volume']>=volume].sort_values(['volume','id'], ascending=[True, True])

    each_order_qualify = list()
    for j in range(box_vol_fitted.shape[0]):

        pre_fit_box = np.sort(box_vol_fitted.iloc[j, 2:5].values)

        if all(pre_fit_box > dims) == True:
            fitted_box_num = box_vol_fitted.iloc[j, 1]
            each_order_qualify.append(fitted_box_num)

    if len(each_order_qualify)>0:
        fitted_box_list.append(each_order_qualify[0])
    else:
        fitted_box_list.append('UNFITTED')

len(fitted_box_list)



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 33608/33608 [07:49<00:00, 71.62it/s]


33608

In [103]:
df_order_multisku = pd.merge(df_order_multisku, pd.DataFrame({
    'order_number': df_order_multisku.order_number.unique(),
    'smallest_box_number_fitted': fitted_box_list
}), on='order_number', how='left')

output_case3 = pd.merge(output_case2, df_order_multisku[['order_number', 'smallest_box_number_fitted']], on='order_number', how='left')
output_case3['box_number'] = np.where(output_case3['box_number']=='UNFITTED', 
                                      output_case3['smallest_box_number_fitted'], 
                                      output_case3['box_number'])
output_case3 = output_case3[['order_number','box_number']]
output_case3['box_number'] = output_case3['box_number'].fillna('UNFITTED')
output_case3 = output_case3.drop_duplicates()
output_case3.shape
output_case3.head()
output_case3.to_csv('submission-retry-20210118.csv', index=False)


(100000, 2)

,order_number,box_number
0,be35924799d0a1b179795420378981820722fd6681eed4...,SP-106
1,fa129fe24ae2fd26ff4ddd132a4f3b2564831e906dd10d...,SP-108
2,05c9ae40ad96a5ab0a527129afbf4a5b28d7bfc8200796...,SP-108
3,643b1bc23521958d9bdd730c85bef211ebc3d11bd93682...,SP-108
4,ea816de880f43e51d95f542d3996a80636aa2e921d4acf...,SP-102


In [101]:
output_case3[output_case3['box_number']=='UNFITTED']

,order_number,box_number
60,029dfc7ce26a9e520a532d238b5a8d406ff8fa83e8873f...,UNFITTED
100,1d38fead305509f0fce933cafaf2b98b823a314cbfc45c...,UNFITTED
195,7ea623d407c8d1042f66e3e84176ae9ba09a06dc493480...,UNFITTED
224,efe41a4ec555f9f740815400858fdc1452897bb962be33...,UNFITTED
242,35118852aad83b52b529e50181d522369062a58952240b...,UNFITTED
...,...,...
180880,4ea8fc06dc6509befe005316f43ff07baba28e4499f9d8...,UNFITTED
180919,c455b317e358d3bb2cbd2f2cce8300d043a66a7d3c4a2d...,UNFITTED
180920,c455b317e358d3bb2cbd2f2cce8300d043a66a7d3c4a2d...,UNFITTED
180952,a06dca2e91f8c12a4c05ecc163062f7e93c67cfa4f5dcb...,UNFITTED
